# Домашнее задание: Работа с свёрточными нейронными сетями на PyTorch

In [140]:
# Для работы с работой установим следующие библиотеки
import numpy as np
from torch import nn
from torch.nn import functional as F
from torch import Tensor

## Задание 1: Операции с матрицами (свёртка)

__В этом задании вам нужно будет реализовать свёртку (convolution) вручную с помощью
матричных операций в PyTorch. Это поможет лучше понять, как работают свёрточные
фильтры.__

### 1. Создайте случайную матрицу (изображение) размером 5x5 и фильтр (ядро свёртки) размером 3x3.

__Случайная матрица 5х5__

In [141]:
img = np.random.randint(0, 256, size=(5, 5))

In [146]:
img

array([[218, 237, 255, 157,  94],
       [143,  50,  18, 156, 128],
       [173,  61, 137, 101, 131],
       [106,  30, 123,  36, 240],
       [ 35,  86,  85, 242,  46]])

__В качестве фильтра создаем следующую матрицу__

In [147]:
kernel = np.zeros((3,3))
kernel[1, 1] = 5
kernel[0, 1] = kernel[1, 0] = kernel[1, 2] = kernel[2, 1] = -1
kernel

array([[ 0., -1.,  0.],
       [-1.,  5., -1.],
       [ 0., -1.,  0.]])

In [154]:
out = np.empty([3,3])

### 2. Реализуйте функцию для выполнения свёртки изображения с этим фильтром. Напишите свёртку с шагом 1 и без использования встроенных функций PyTorch для свёртки

In [155]:
def conv(img: np.ndarray, kernel: np.ndarray, stride: int=1) -> np.ndarray:
    # т.к. в данной функции учитываем только шаг фильтра,
    #то формулы размеров выходной матрицы равны:
    h_in = img.shape[0]
    w_in = img.shape[1]
    h_out = int(((h_in - (kernel.shape[0] - 1) - 1) / stride) + 1)
    w_out = int(((w_in - (kernel.shape[1] - 1) - 1) / stride) + 1)
    # Создаем пустой массив для свертки, размером, определнным выше
    out = np.empty([h_out, w_out])
    # Проходим ядром по матрице
    for i in range(h_out):
        for j in range(w_out):
            out[i, j] = np.sum(kernel * img[i:h_out+i, j:w_out+j])

# 0
#     0 - out[0,0] - img[0:3, 0:3]
#     1 - out[0,1] - img[0:3, 1:4]
#     2 - out[0,2] - img[0:3, 2:5]
# 1
#     0 - out[1,0] - img[1:4, 0:3]
#     1 - out[1,1] - img[1:4, 1:4]
#     2 - out[1,2] - img[1:4, 2:5]
# 2
#     0 - out[1,0] - img[2:5, 0:3]
#     1 - out[1,1] - img[2:5, 1:4]
#     2 - out[1,2] - img[2:5, 2:5]
    
    return out
conv(img, kernel, 1)

array([[-209., -508.,  376.],
       [ -85.,  382.,   45.],
       [-226.,  327., -526.]])

### 3. Проверьте работу вашей функции, сравнив результат с использованием встроенной функции torch.nn.functional.conv2d.

Для использования функции torch.nn.functional.conv2d необходимо преобразовать матрицы входа и ядра в тензоры. Для этого используем функцию reshape, где зададим количествовходных и выходных каналов, равными 1, а рзмеры тензоров, равными размерами матриц.

In [157]:
in_t = Tensor(img).reshape(1, 1, img.shape[0], img.shape[1])
in_w = Tensor(kernel).reshape(1, 1, kernel.shape[0], kernel.shape[1])
out_torch = F.conv2d(in_t, in_w)

In [158]:
out_torch

tensor([[[[-209., -508.,  376.],
          [ -85.,  382.,   45.],
          [-226.,  327., -526.]]]])

__Как видно, результаты написанной функции и результаты подстановки в функцию torch.nn.functional.conv2d совпадают.__

## Задание 2: Настройка количества параметров между слоями свёртки

__Во втором задании вам нужно создать простую свёрточную нейронную сеть с несколькими слоями и настроить количество фильтров (параметров) между слоями свёртки, а также применить паддинг (padding), чтобы избежать уменьшения размерности данных.__

### Создайте модель с двумя свёрточными слоями с использованием torch.nn.Conv2d.
* Первый свёрточный слой должен содержать 16 фильтров, второй — 32 фильтра.
* Для сохранения размерности используйте паддинг 1.
* После свёрточных слоёв добавьте слой выравнивания (flatten) и полносвязный слой (fully connected).

In [159]:
class MODEL(nn.Module):
    def __init__(self):
        super(MODEL, self).__init__()
        # сверточные слои
        self.conv_layers = nn.Sequential(
            nn.Conv2d(1, 16, padding=1, kernel_size=3),
            nn.Conv2d(16, 32, padding=1, kernel_size=3),
        )
        # слой выравнивания
        self.flatten_layer =  nn.Sequential(
            nn.Flatten())
        # полносвязный слой - зададим количество классов, равным 2, размеры тензора, равными размеру матрицы, т.к. используем padding=1
        self.fc_layer = nn.Sequential(
            nn.Linear(32 * img.shape[0] * img.shape[1], 2)
        )
    
    def forward(self, x):
        x = self.conv_layers(x)
        x = self.flatten_layer(x)
        x = x.view(x.size(0), -1)
        x = self.fc_layer(x)
        return x

__Использование модели__

In [160]:
model = MODEL()
print(model(in_t))

tensor([[12.2007, 16.2408]], grad_fn=<AddmmBackward0>)


## Задание 3: Использование различных шагов и паддингов

В этом задании вам предстоит поэкспериментировать с различными значениями шага (stride) и паддинга (padding), а также объяснить, как они влияют на выходные размеры слоёв свёртки.

### 1. Создайте три свёрточных слоя, каждый из которых будет использовать разные значения шага и паддинга:
* Первый слой с шагом 1 и паддингом 1.
* Второй слой с шагом 2 и без паддинга.
* Третий слой с шагом 2 и паддингом 2.

Количество фильтров для наглядности берем 2, чтобы получить компактный вывод тензора.

__Первый слой с шагом 1 и паддингом 1__

In [161]:
conv_1 = nn.Conv2d(1, 2, stride=1, padding=1, kernel_size=3)
conv_out_1 = conv_1(in_t)

In [165]:
conv_out_1

tensor([[[[ 4.8881e+01,  4.2643e+01,  5.6469e+01,  7.3015e+01,  4.9126e+01],
          [ 5.0440e+01,  7.6963e+01,  5.2033e+01,  7.8471e+01,  5.5196e+01],
          [ 1.8074e+01,  5.0482e+01,  7.5947e+01,  4.6839e+01,  6.4992e+01],
          [ 6.0351e-01,  7.2211e+01,  6.2750e+01,  8.2630e+01,  3.2562e+01],
          [-1.7876e+01,  3.9372e+01, -1.2620e+01,  8.1229e+01, -3.2039e+01]],

         [[ 4.5890e+01, -1.3271e+01, -6.0293e+01, -1.6543e+01, -3.7983e+00],
          [-3.9119e+01, -1.2956e+02, -4.0449e+01, -6.6419e+01, -6.7681e+01],
          [-1.8935e+01, -3.8248e+01, -3.3595e+01, -3.0621e+01, -3.3319e+01],
          [-3.1095e+01, -4.8353e+01, -3.3929e+01,  3.5921e+01, -7.1848e+01],
          [-1.2206e-01, -6.8562e+01, -1.6611e+00, -1.6418e+02, -8.5494e+01]]]],
       grad_fn=<ConvolutionBackward0>)

__Второй слой с шагом 2 и без паддинга__

In [166]:
conv_2 = nn.Conv2d(1, 2, stride=2, padding=0, kernel_size=3)
conv_out_2 = conv_2(in_t)

In [167]:
conv_out_2

tensor([[[[ 87.6905,  42.5465],
          [ 77.3349,  62.5965]],

         [[176.0463,  90.9090],
          [107.7828, 107.9515]]]], grad_fn=<ConvolutionBackward0>)

__Третий слой с шагом 2 и паддингом 2__

In [168]:
conv_3 = nn.Conv2d(1, 2, stride=2, padding=2, kernel_size=3)
conv_out_3 = conv_3(in_t)

In [169]:
conv_out_3

tensor([[[[ -18.5900,   87.7835,   83.0492,   15.6753],
          [ -11.0188,  187.7864,   59.7333,   47.1579],
          [   2.3548,   53.4834,   88.6277,   55.0275],
          [   8.0414,   46.9016,   87.1116,   -0.2006]],

         [[ -26.8952, -119.1813,  -95.0545,  -21.6799],
          [ -66.4907,  -85.1178,  -30.0258,  -45.3721],
          [ -39.5898,  -57.2589, -104.8246,  -44.8756],
          [  -5.1875,   -2.7383,   18.1179,    5.2139]]]],
       grad_fn=<ConvolutionBackward0>)

### 2. Для каждого слоя выведите размерность выходных данных и объясните, как шаг и паддинг влияют на размер изображения.

In [173]:
# Размер свертки первого слоя (шаг=1, паддинг=1)
conv_out_1.shape

torch.Size([1, 2, 5, 5])

In [174]:
# Размер свертки второго слоя (шаг=2, без паддинга)
conv_out_2.shape

torch.Size([1, 2, 2, 2])

In [175]:
# Размер свертки третьего слоя (шаг=2, паддинг=2)
conv_out_3.shape

torch.Size([1, 2, 4, 4])

Расчет размера свертки также можно произвести методами Python, используя формулу вычисления размера свертки:

In [176]:
def conv_calc(data, k, p, d, s):
    
    '''
    data - размерность входного тензора
    k - размер ядра [int, int]
    p - паддинг [int, int]
    d - расширение [int, int] (используются значения по умолчанию в torch - (1, 1))
    s - шаг ядра [int, int] (используются значения по умолчанию в torch - (1, 1))
    '''
    
    h_out = ((data[0] + 2 * p - d[0] * (k[0] - 1) - 1) / s[0]) + 1
    w_out = ((data[1] + 2 * p - d[1] * (k[1] - 1) - 1) / s[1]) + 1
    
    return (int(h_out), int(w_out))

In [177]:
# для первого слоя (шаг=1, паддинг=1)
p = 1
d = (1, 1)
s = (1, 1)
print(f'Размер свертки 1-го слоя {conv_calc(img.shape, kernel.shape, p, d, s)}')

Размер свертки 1-го слоя (5, 5)


In [178]:
# для второго слоя (шаг=2, без паддинга)
p = 0
d = (1, 1)
s = (2, 2)
print(f'Размер свертки 1-го слоя {conv_calc(img.shape, kernel.shape, p, d, s)}')

Размер свертки 1-го слоя (2, 2)


In [179]:
# для третьего слоя (шаг=2, паддинг=2)
p = 2
d = (1, 1)
s = (2, 2)
print(f'Размер свертки 1-го слоя {conv_calc(img.shape, kernel.shape, p, d, s)}')

Размер свертки 1-го слоя (4, 4)


__Задание: После выполнения задания опишите, как каждый слой меняет размерность данных, и каким образом влияют параметры шагов и паддингов__

__Слой 1__ - не меняет размерность данных, то есть шаг=1 и паддинг=1 как бы нивелируют друг друга.

__Слой 2__ - паддинг отсутствует, соответственно, размерность данных уменьшается, а шаг=2 ускоряет этот процесс.

__Слой 3__ - шаг=2, как в предыдущем слое значительно сокращает размерность данных, однако паддинг=2 ее увеличивает.

Из проведенных исследованийи формулы, реализованной в функции conv_calc, можне заключить следующее:
1. Паддинг (p): одна единица паддинга увеличивает тензор в ширину и высоту на 2 единицы (при шаге=1; если шаг больше 1, то увеличение размерности условно можно принять как удвоенное отношение паддинга к шагу).
2. Шаг (s): увеличение шага приводит к уменьшению выходного тензора. Размерность выходных данных условно в s раз меньше, чем входных (т.к. в формуле подсчета размерности присутствует +1 в самом конце).